In [1]:
import pandas as pd

In [2]:
# Paso 1: Leer informe Excel Clientes
f1 = pd.DataFrame(columns=['CEDULA', 'Nombre y apellido'])
#f1 = pd.read_excel("archivos2/BASE SOCIAL A 31 DE DICIEMBRE DE 2023 APORTES Y CREDITOS.xlsx")
#f1 = pd.read_excel("archivos/BASE DE DATOS AÑO 2023 INGRESOS Y EGRESOS.xlsx")
f1 = pd.read_excel("ARCHIVOS5/BASE DE DATOS CLIENTES 25-04-2024.xlsx")
f1.columns = [i.strip() for i in f1.columns] # Removiendo espacios en columnas
f1 = f1.rename(columns={"NIT":"CEDULA", "Cedula":"CEDULA", "NOMBREINTE":"Nombre y apellido", "AGENCIA":"COD AGENCIA"})
f1["CEDULA"] = f1["CEDULA"].astype('int64')  # Convirtiendo cédula a entero
f1

,CEDULA,Nombre y apellido,DIRECCION,RELACION,COD AGENCIA,TELEFONO1,EXTENSION1,TELEFONO2,EXTENCION2,FECHAINGRE,...,CODBEEPER,CELULAR,NOMBREPAIS,NOMBREDEPA,NOMBRECIUD,NOMBREAGEN,ESTADO,SEXO,ESTADOCIVI,NACIONALID
0,211134,FOX FREDERICK PUTNAM,CR 8 # 25-53 APX 301,A,1,0,NaN,0,NaN,2016-02-29,...,NaN,312 2281552,COLOMBIA,CHOCO,QUIBDO,PRINCIPAL,Activo,MASCULINO,CASADO(A),NaN
1,519043,RAMOS ARIAS RUFFO RODRIGO,CR 3 N. 29-51,A,1,6 71 13 63,NaN,NaN,NaN,2011-10-01,...,NaN,314 8351709,COLOMBIA,CHOCO,QUIBDO,PRINCIPAL,Activo,MASCULINO,CASADO(A),NaN
2,1272726,RENTERIA MACHADO TULIO ARIEL,CL 22 N 17 23,A,1,0,NaN,NaN,NaN,2016-01-07,...,NaN,321 7019855,COLOMBIA,CHOCO,QUIBDO,PRINCIPAL,Activo,MASCULINO,CASADO(A),NaN
3,1582662,PALACIOS ANGEL ANTONIO,CL 16 N 6-12 PABLO VI,A,1,6 71 36 32,NaN,NaN,NaN,2013-02-06,...,NaN,314 6240351,COLOMBIA,CHOCO,QUIBDO,PRINCIPAL,Activo,MASCULINO,CASADO(A),NaN
4,1584577,BEJARANO HURTADO JUVENAL,CR 1 N 36-22 BRR KENNEDY,A,1,0,0,0,NaN,2014-03-18,...,NaN,311 3205015,COLOMBIA,CHOCO,QUIBDO,PRINCIPAL,Activo,MASCULINO,UNION LIBRE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11228,9002332501,CONCEJO MUNICIPAL DE SIPI,BRR MUNICIPIO DE SIPI,T,2,NaN,NaN,NaN,NaN,2020-10-01,...,NaN,310 8152704,COLOMBIA,CHOCO,SIPI,AGENCIA ISTMINA,Activo,NaN,NaN,NaN
11229,9020161279,BEBIDAS Y ALIMENTOS ALAMEDA S.A.,CL 26 N 7-35 ALAMEDA REYES,T,2,0 0,NaN,NaN,NaN,2014-04-30,...,NaN,NaN,COLOMBIA,CHOCO,QUIBDO,AGENCIA ISTMINA,Activo,NaN,NaN,NaN
11230,95041724081,VILLAREAL TORRES YEFFERSON ALBERTO,BRR PUEBLO NUEVO,T,2,0,NaN,NaN,NaN,2012-04-03,...,NaN,321 7316755,COLOMBIA,CHOCO,ISTMINA,AGENCIA ISTMINA,Fallecido,MASCULINO,SOLTERO(A),NaN
11231,96012713042,VENTURA HURTADO MARLON ANDRES,VDA ANIMAS EL DOS,T,2,0 0,NaN,NaN,NaN,2013-07-06,...,NaN,310 4268693,COLOMBIA,CHOCO,UNION PANAMERICANA,AGENCIA ISTMINA,Activo,MASCULINO,SOLTERO(A),NaN


In [3]:
# Paso 2: Leer informe Excel Transacciones
f2 = pd.read_excel("ARCHIVOS5/Febrero 2024 Afroamericana OPA.xlsx")
#f2 = pd.read_excel("archivos/BD VISIONAMOS.xlsx")
#f2 = pd.read_excel("archivos2/TRANSACCIONES VISIONAMOS.xlsx")
f2

,Fecha,Cedula,Documento,Codagencia,Codlinea,Naturaleza,Totalefectivo,Codcuenta,Movcuenta,Descripción
0,2024-02-01 08:04:22.000,54257726,01 12720,1,ABNA,CNGC,100000.0,21250506,C,PAGO CUOTA DE AFROBONO
1,2024-02-01 08:16:43.000,18532346,23 14732,2,AHVI,RETC,43000000.0,21050505,R,CONSIGNACION AHORRO
2,2024-02-01 08:16:46.000,18532346,23 14732,2,AHVI,RETC,172000.0,21050505,R,CONSIGNACION AHORRO
3,2024-02-01 08:28:51.000,3396826,01 12720,1,AHVI,CNGC,200000.0,21050505,C,CONSIGNACION AHORRO
4,2024-02-01 08:52:23.000,26257633,01 12720,1,AHVI,CNGC,350000.0,21050505,C,CONSIGNACION AHORRO
...,...,...,...,...,...,...,...,...,...,...
1054,2024-02-29 21:05:08.000,1020241494,01 12788,1,AHVM,CNGC,20000.0,21050510,C,APERTURA DE CUENTA DE AHORRO
1055,2024-02-29 21:13:35.000,1077451584,01 12788,1,AHVI,CNGC,20000.0,21050505,C,APERTURA DE CUENTA DE AHORRO
1056,2024-02-29 21:15:14.000,35603366,01 12788,1,AHVI,CNGC,20000.0,21050505,C,APERTURA DE CUENTA DE AHORRO
1057,2024-02-29 21:17:30.000,4851978,01 12788,1,AHVI,CNGC,20000.0,21050505,C,APERTURA DE CUENTA DE AHORROS


In [4]:
f2.columns = [i.strip() for i in f2.columns] # Removiendo espacios en columnas
f2.columns = [col.upper() for col in f2.columns]
f2 = f2.rename(columns={"CEDULA":"DOCUMENTO1", "TOTAL EFECTIVO":"VALOR", "TIPO DE MOVIMIENTO":"OPERACION",
                       "AGENCIA": "COD. AGENCIA", "TIPODEMOVIMIENTO":"OPERACION", "TOTALEFECTIVO": "VALOR",
                       "NATURALEZA":"OPERACION", "CODAGENCIA":"COD. AGENCIA", "CODLINEA":"PRODUCTO"})

f2["OPERACION"] = f2["OPERACION"].apply(lambda x: "CREDITO" if "CNGC" in x else "DEBITO")
f2["OPERACION"] = f2["OPERACION"].apply(lambda x: "CREDITO" if "C" in x else "DEBITO")
if 'DISPOSITIVO' not in f2.columns: f2['DISPOSITIVO']=''
f2 = f2.rename(columns={'FECHA_REGISTRO':'FECHA'})
if "ESTADO" in f2.columns: f2 = f2[f2["ESTADO"]=="APROBADA"]
if 'FECHA_REGISTRO' in f2.columns: 
    f2["FECHA"] = pd.to_datetime(f2['FECHA'], format=formato) 
else:
    f2["FECHA"] = pd.to_datetime(f2['FECHA'])  
    
f2['Mes'] = f2['FECHA'].dt.to_period('M')  # Extrayendo el mes de la fecha
f2["DOCUMENTO1"] = f2["DOCUMENTO1"].astype('int64', errors='ignore') # Convirtiendo documento a entero
f2['CANAL'] = f2['CANAL'] + ' ' + f2['DISPOSITIVO'] if 'CANAL' in f2 else ''
condAgencia = 1 if "COD. AGENCIA" in f2.columns else 0
producto = 1 if "PRODUCTO" in f2.columns else 0
f2

,FECHA,DOCUMENTO1,DOCUMENTO,COD. AGENCIA,PRODUCTO,OPERACION,VALOR,CODCUENTA,MOVCUENTA,DESCRIPCIÓN,DISPOSITIVO,Mes,CANAL
0,2024-02-01 08:04:22,54257726,01 12720,1,ABNA,CREDITO,100000.0,21250506,C,PAGO CUOTA DE AFROBONO,,2024-02,
1,2024-02-01 08:16:43,18532346,23 14732,2,AHVI,DEBITO,43000000.0,21050505,R,CONSIGNACION AHORRO,,2024-02,
2,2024-02-01 08:16:46,18532346,23 14732,2,AHVI,DEBITO,172000.0,21050505,R,CONSIGNACION AHORRO,,2024-02,
3,2024-02-01 08:28:51,3396826,01 12720,1,AHVI,CREDITO,200000.0,21050505,C,CONSIGNACION AHORRO,,2024-02,
4,2024-02-01 08:52:23,26257633,01 12720,1,AHVI,CREDITO,350000.0,21050505,C,CONSIGNACION AHORRO,,2024-02,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,2024-02-29 21:05:08,1020241494,01 12788,1,AHVM,CREDITO,20000.0,21050510,C,APERTURA DE CUENTA DE AHORRO,,2024-02,
1055,2024-02-29 21:13:35,1077451584,01 12788,1,AHVI,CREDITO,20000.0,21050505,C,APERTURA DE CUENTA DE AHORRO,,2024-02,
1056,2024-02-29 21:15:14,35603366,01 12788,1,AHVI,CREDITO,20000.0,21050505,C,APERTURA DE CUENTA DE AHORRO,,2024-02,
1057,2024-02-29 21:17:30,4851978,01 12788,1,AHVI,CREDITO,20000.0,21050505,C,APERTURA DE CUENTA DE AHORROS,,2024-02,


In [5]:
# Paso 3: Extrayendo datos de clientes
b1 = f1[["CEDULA", "Nombre y apellido"]].copy()
for col in ["Ingresos", "Egresos", "Activos", "Pasivos"]:
    b1[col] = f1[col].values if col in f1.columns else 0
b1["Codigo"] = 0 # Código actividad económica: CERO
b1 = b1.fillna(0)
b1.to_excel("Clientes.xlsx", index=False)
b1

,CEDULA,Nombre y apellido,Ingresos,Egresos,Activos,Pasivos,Codigo
0,211134,FOX FREDERICK PUTNAM,0,0,0,0,0
1,519043,RAMOS ARIAS RUFFO RODRIGO,0,0,0,0,0
2,1272726,RENTERIA MACHADO TULIO ARIEL,0,0,0,0,0
3,1582662,PALACIOS ANGEL ANTONIO,0,0,0,0,0
4,1584577,BEJARANO HURTADO JUVENAL,0,0,0,0,0
...,...,...,...,...,...,...,...
11228,9002332501,CONCEJO MUNICIPAL DE SIPI,0,0,0,0,0
11229,9020161279,BEBIDAS Y ALIMENTOS ALAMEDA S.A.,0,0,0,0,0
11230,95041724081,VILLAREAL TORRES YEFFERSON ALBERTO,0,0,0,0,0
11231,96012713042,VENTURA HURTADO MARLON ANDRES,0,0,0,0,0


In [6]:
# Paso 4: Extrayendo datos de transacciones
cols = ['DOCUMENTO1', 'Mes', 'OPERACION', 'CANAL', 'VALOR']
if condAgencia: cols=cols[:-1]+["COD. AGENCIA", "VALOR"]
if producto: cols=cols[:-1]+["PRODUCTO", "VALOR"]
f2 = f2[cols] # Extrayendo columnas

# Agrupando por documento, mes, operacion y canal: (Hallando conteo y suma del valor)
b2 = f2.groupby(cols[:-1]).agg({'count', 'sum'}).reset_index()
b2.columns = b2.columns.droplevel(level=0)

# Renombrando nuevamente las columnas (Por la agrupacion se borraron los nombres)
b2.columns.values[:len(cols)-1] = cols[:-1]
b2 = b2.rename(columns={"sum":"VALOR", "count":"N_VALOR"})

# Creando variable productos
if not producto:
    b2["PRODUCTO"] = "Ahorros"
b2["DOCUMENTO1"] = b2["DOCUMENTO1"].astype('int64') # Convirtiendo documento a entero

# Agregando columna jurisdiccion (COD. AGENCIA) a los datos desde clientes
if condAgencia==0:
    b2 = pd.merge(b2, f1[['CEDULA', 'COD. AGENCIA']], left_on='DOCUMENTO1', right_on='CEDULA', how='left')
    b2 = b2.drop("CEDULA", axis=1)
b2 = b2.fillna("*")

# Renombrar columnas debito y creditos
columns = ["Mes", "DOCUMENTO1", "N_VALOR", "VALOR", "PRODUCTO", 'CANAL', "COD. AGENCIA"]
b21 = b2[b2["OPERACION"]=="DEBITO"][columns].rename(columns={"Mes":"Fecha", "DOCUMENTO1":"CEDULA", 
                                                             'CANAL':"CANAL", "N_VALOR":"N_DEBITO",
                                                             "VALOR":"SUMA_DEBITO", "COD. AGENCIA":"JURISDICCION"})

b22 = b2[b2["OPERACION"]=="CREDITO"][columns].rename(columns={"Mes":"Fecha", "DOCUMENTO1":"CEDULA", 
                                                              'CANAL':"CANAL", "N_VALOR":"N_CREDITO",
                                                              "VALOR":"SUMA_CREDITO", "COD. AGENCIA":"JURISDICCION"})


# Concatenando columnas debito, creditos y ordenando
b2 = pd.concat([b21, b22]).fillna(0).sort_values(by=["CEDULA", "Fecha"])
b2 = b2[["Fecha", "CEDULA", "N_DEBITO", "SUMA_DEBITO", "N_CREDITO", 
         "SUMA_CREDITO", "PRODUCTO", "CANAL", "JURISDICCION"]]
b2.to_excel("Transacciones.xlsx", index=False)
b2

,Fecha,CEDULA,N_DEBITO,SUMA_DEBITO,N_CREDITO,SUMA_CREDITO,PRODUCTO,CANAL,JURISDICCION
0,2024-02,3396826,0.0,0.0,1.0,200000.0,AHVI,,1
2,2024-02,4797540,3.0,2582250.0,0.0,0.0,AHVI,,1
1,2024-02,4797540,0.0,0.0,1.0,72250.0,APSO,,1
3,2024-02,4800579,3.0,2510754.0,0.0,0.0,AHVI,,1
4,2024-02,4809228,2.0,4668600.0,0.0,0.0,AHVI,,1
...,...,...,...,...,...,...,...,...,...
454,2024-02,1149441372,0.0,0.0,2.0,1358000.0,AHVI,,2
457,2024-02,1186463024,1.0,50000.0,0.0,0.0,AHVI,,1
456,2024-02,1186463024,0.0,0.0,1.0,50000.0,ABNA,,1
458,2024-02,1193088419,0.0,0.0,1.0,20000.0,AHVI,,1
